## Test Docling Image Description :

In [ ]:
from docling.datamodel.pipeline_options import granite_picture_description
from docling.datamodel.pipeline_options import smolvlm_picture_description
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat

In [2]:
pipeline_options = PdfPipelineOptions()
pipeline_options.do_picture_description = True
pipeline_options.picture_description_options = (
    smolvlm_picture_description  # <-- the model choice
)
pipeline_options.picture_description_options.prompt = (
    "Describe the image in three sentences. Be consise and accurate."
)
pipeline_options.images_scale = 2.0
pipeline_options.generate_picture_images = True

converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
            pipeline_options=pipeline_options,
        )
    }
)

In [3]:
doc = converter.convert("https://arxiv.org/pdf/2501.17887").document # Doc source path

[INFO] 2026-02-06 21:29:16,595 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-06 21:29:16,604 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-06 21:29:16,652 [RapidOCR] download_file.py:60: File exists and is valid: C:\IMON\Masters\Self-Learning\saas-rag-app\my_venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-06 21:29:16,653 [RapidOCR] main.py:50: Using C:\IMON\Masters\Self-Learning\saas-rag-app\my_venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-06 21:29:16,841 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-06 21:29:16,841 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-06 21:29:16,852 [RapidOCR] download_file.py:60: File exists and is valid: C:\IMON\Masters\Self-Learning\saas-rag-app\my_venv\Lib\site-packages\rapidocr\models\ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-02-06 21:29:16,853 [RapidOCR] main.py:50: Using C:\IMON\Masters\Self

In [3]:
doc = converter.convert("C:\\IMON\\Masters\\Self-Learning\\saas-rag-app\\api\\documents\\Seminar PPT-V2.pptx").document # Doc source path

In [4]:
markdown_content = doc.export_to_markdown()

In [5]:
output_path = "C:\\IMON\\Masters\\Self-Learning\\saas-rag-app\\api\\documents\\presentation.md" # Path to save markdown content
with open(output_path, "w", encoding="utf-8") as f:
    f.write(markdown_content)

## Test Supabase Queries :

In [1]:
import logging
from dotenv import load_dotenv
import asyncio
import asyncpg
import os

In [2]:
# Load environment variables
load_dotenv()

logger = logging.getLogger(__name__)
# Global database pool
db_pool = None

In [3]:
async def initialize_db():
    """Initialize database connection pool."""
    global db_pool
    if not db_pool:
        db_pool = await asyncpg.create_pool(
            os.getenv("DATABASE_URL"),
            min_size=2,
            max_size=10,
            command_timeout=60
        )
        logger.info("Database connection pool initialized")

In [4]:
await initialize_db()

In [6]:
async with db_pool.acquire() as conn:
    results = await conn.fetch(
        """
        SELECT title
        FROM "documents"
        """,
    )

In [13]:
results

[<Record title='q4-2024-business-review'>,
 <Record title='14 types of RAG (Retrieval-Augmented Generation)'>,
 <Record title='Recording2'>,
 <Record title='Recording1'>,
 <Record title='Recording3'>,
 <Record title='Deep Learning for Precipitation Nowcasting'>,
 <Record title='Docling: An Efficient Open-Source Toolkit for AI-driven Document Conversion'>]

In [15]:
titles = [r["title"] for r in results if r["title"] is not None]
titles

['q4-2024-business-review',
 '14 types of RAG (Retrieval-Augmented Generation)',
 'Recording2',
 'Recording1',
 'Recording3',
 'Deep Learning for Precipitation Nowcasting',
 'Docling: An Efficient Open-Source Toolkit for AI-driven Document Conversion']

In [ ]:
async with db_pool.acquire() as conn:
    results = await conn.fetch(
        """
        SELECT chunk_index, content
        FROM chunks
        WHERE metadata->>'title' = $1
        ORDER BY chunk_index
        """,
        "Docling: An Efficient Open-Source Toolkit for AI-driven Document Conversion",  # e.g. "Docling: An Efficient Open-Source Toolkit for AI-driven Document Conversion"
    )


In [17]:
results

[<Record chunk_index=0 content='Docling: An Efficient Open-Source Toolkit for AI-driven Document Conversion\nNikolaos Livathinos * , Christoph Auer * , Maksym Lysak, Ahmed Nassar, Michele Dolfi, Panagiotis Vagenas, Cesar Berrospi, Matteo Omenetti, Kasper Dinkla, Yusik Kim, Shubham Gupta, Rafael Teixeira de Lima, Valery Weber, Lucas Morin, Ingmar Meijer, Viktor Kuropiatnyk, Peter W. J. Staar\nIBM Research, R¨ uschlikon, Switzerland\nPlease send correspondence to: deepsearch-core@zurich.ibm.com'>,
 <Record chunk_index=1 content="Abstract\nWe introduce Docling , an easy-to-use, self-contained, MITlicensed, open-source toolkit for document conversion, that can parse several types of popular document formats into a unified, richly structured representation. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. Docling is released as a Python

In [18]:
full_content = "\n".join(
    r["content"] for r in sorted(results, key=lambda x: x["chunk_index"])
    if r["content"]
)

In [19]:
print(full_content)

Docling: An Efficient Open-Source Toolkit for AI-driven Document Conversion
Nikolaos Livathinos * , Christoph Auer * , Maksym Lysak, Ahmed Nassar, Michele Dolfi, Panagiotis Vagenas, Cesar Berrospi, Matteo Omenetti, Kasper Dinkla, Yusik Kim, Shubham Gupta, Rafael Teixeira de Lima, Valery Weber, Lucas Morin, Ingmar Meijer, Viktor Kuropiatnyk, Peter W. J. Staar
IBM Research, R¨ uschlikon, Switzerland
Please send correspondence to: deepsearch-core@zurich.ibm.com
Abstract
We introduce Docling , an easy-to-use, self-contained, MITlicensed, open-source toolkit for document conversion, that can parse several types of popular document formats into a unified, richly structured representation. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. Docling is released as a Python package and can be used as a Python API or as a CLI tool. Docling's mo

In [ ]:
async with db_pool.acquire() as conn:
    async with conn.transaction():
        await conn.execute(
            "DELETE FROM chunks WHERE metadata->>'title' = $1",
            "API Gateway Configuration Example gateway: host: api.neuralflow-ai.com port: 443 ssl: true rate_limit: requests_per_minute: 1000 burst: 100 auth: type: jwt token_expiry: 3600 routes: - path: /v1/documents/* service: document-processor methods: [POST, GET] - path: /v1/chat/* service: document-processor methods: [POST, GET] - path: /v1/chat/* service: conversational-ai methods: [POST, GET, DELETE]",
        )
        await conn.execute(
            "DELETE FROM documents WHERE title = $1",
            "NeuralFlow AI - Mission and Goals",
        )


In [8]:
titles = ["NeuralFlow AI - Team Handbook","Boids / Flocking Algorithm"]

async with db_pool.acquire() as conn:
    async with conn.transaction():
        # 1) Delete all chunks whose metadata->>'title' matches any of the titles
        await conn.execute(
            """
            DELETE FROM chunks
            WHERE metadata->>'title' = ANY($1::text[])
            """,
            titles,
        )

        # 2) Delete all documents whose title matches any of the titles
        await conn.execute(
            """
            DELETE FROM documents
            WHERE title = ANY($1::text[])
            """,
            titles,
        )
